### Imports + helpers

In [28]:
import re
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil import parser as dateparser
import pandas as pd

START_URL = "https://www.sharepointeurope.com/conference/schedule/2025-Fabric/"
BASE_URL  = "https://www.sharepointeurope.com"
CONFERENCE = "FabCon2025EuropeVienna"
CONF_CODE = 1000
CONF_PREFIX = "2025_Europe"
YEAR_HINT  = 2025

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115 Safari/537.36"
}

StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 30, Finished, Available, Finished)

In [29]:
def get_soup(url: str) -> BeautifulSoup:
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")

def clean(s: str | None) -> str | None:
    if not s:
        return None
    # normalize all whitespace (incl. non-breaking) to single spaces
    s = s.replace("\xa0", " ")
    return re.sub(r"\s+", " ", s).strip()

def abs_url(href: str | None) -> str | None:
    return urljoin(BASE_URL, href) if href else None

def normalize_split_digits(day_str: str) -> str:
    """
    Fix cases like '1 5' -> '15' that come from split text nodes.
    Only collapse spaces that are BETWEEN digits; leave other spaces intact.
    """
    if not day_str:
        return day_str
    # Handle any unicode whitespace between digits
    return re.sub(r'(?<=\d)\s+(?=\d)', '', day_str)

StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 31, Finished, Available, Finished)

In [38]:
from datetime import datetime, date, time

ID_TIME_RE = re.compile(r"(\d{2})(\d{2})[-_](\d{2})(\d{2})$")
TIME_WINDOW_RE = re.compile(r"(\d{1,2}:\d{2})\s*[-–]\s*(\d{1,2}:\d{2})")

def parse_header_times(group_header_tbody, *, day_hint: date, clean=lambda s: " ".join((s or "").split())):
    """
    Parse start/end datetimes for a time-group header.

    RULES:
      - Always trust the `day_hint` (from the enclosing day block) for the DATE.
      - Only extract the TIMES from either:
          (a) the compact id suffix (HHMM-HHMM), or
          (b) visible text like "HH:MM - HH:MM".
    """
    if not isinstance(day_hint, date):
        return None, None

    # 1) Try compact id
    hdr_id = (group_header_tbody.get("id") or "").strip()
    m_id = ID_TIME_RE.search(hdr_id)
    start_tm = end_tm = None
    if m_id:
        sh, sm, eh, em = m_id.groups()
        start_tm = datetime.strptime(f"{sh}:{sm}", "%H:%M").time()
        end_tm   = datetime.strptime(f"{eh}:{em}", "%H:%M").time()

    # 2) Fallback to visible text
    if not (start_tm and end_tm):
        time_text = clean(group_header_tbody.get_text(" "))
        m2 = TIME_WINDOW_RE.search(time_text or "")
        if not m2:
            return None, None
        start_tm = datetime.strptime(m2.group(1), "%H:%M").time()
        end_tm   = datetime.strptime(m2.group(2), "%H:%M").time()

    # 3) Combine with the trusted day
    start_dt = datetime.combine(day_hint, start_tm)
    end_dt   = datetime.combine(day_hint, end_tm)
    return start_dt, end_dt


StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 40, Finished, Available, Finished)

In [37]:
def parse_group_sessions(group_table, day_hint):
    """
    Inside a <table class="tb-group-block tb-events-block">:
      - a <tbody class="group-header" id="YYYY-MM-DDHHMM-HHMM"> header
      - followed by one or more <tbody class="group-session"> blocks
    """
    rows = []
    tbodys = group_table.find_all("tbody", recursive=False)
    current_start = current_end = None

    i = 0
    while i < len(tbodys):
        tb = tbodys[i]
        classes = " ".join(tb.get("class", []))

        if "group-header" in classes:
            current_start, current_end = parse_header_times(tb, day_hint=day_hint)
            i += 1
            continue

        if "group-session" in classes:
            # First row has code/track/title/level/topic
            first_tr = tb.find("tr")
            if first_tr:
                # Code (A, B, T1…)
                code_td = first_tr.find("td", class_="code")
                code = clean(code_td.get_text()) if code_td else None

                # Track/category (use data-groupname if present)
                tag_td = first_tr.find("td", class_=lambda c: c and "tag" in c.split())
                track = None
                if tag_td:
                    track = tag_td.get("data-groupname") or clean(tag_td.get_text())

                # Title + link
                title_p = first_tr.find("p", class_="event-title")
                title = None
                link = None
                if title_p:
                    a = title_p.find("a", href=True)
                    if a:
                        title = clean(a.get_text())
                        link  = abs_url(a["href"])

                # Level
                lvl_td = first_tr.find("td", class_=lambda c: c and "level-cell" in c.split())
                level = clean(lvl_td.get_text()) if lvl_td else None

                # Topic
                topic_td = first_tr.find("td", class_=lambda c: c and "topic-cell" in c.split())
                topic = clean(topic_td.get_text()) if topic_td else None

                # Speakers row is often a following <tr class="meta"> in the SAME tbody
                meta_tr = tb.find("tr", class_="meta")
                speakers = None
                if meta_tr:
                    anchor_names = [clean(a.get_text()) for a in meta_tr.find_all("a", href=True)]
                    anchor_names = [x for x in anchor_names if x]
                    speakers = "; ".join(anchor_names) if anchor_names else None

                rows.append({
                    "title": title,
                    "detail_url": link,
                    "start": current_start,
                    "end": current_end,
                    "level": level,
                    "topic": topic,
                    "track": track,
                    "code": code,
                    "speakers": speakers
                })
            i += 1
            continue

        # Any other tbody types — skip
        i += 1

    return rows


StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 39, Finished, Available, Finished)

### Get the schedule

In [39]:
def extract_schedule(url=START_URL):
    soup = get_soup(url)
    all_rows = []

    # Iterate every day block
    for day_div in soup.select("div.tb-day"):

        day_title_el = (
            day_div.select_one("table.tb-day-block tbody.group-day p")
            or day_div.find(class_=re.compile(r"(tb-day-title|tb-day-name|day-title)"))
            or day_div.find(["h1", "h2", "h3", "header"])
        )
        day_str = (day_title_el.get_text(" ") if day_title_el else "").replace("\xa0", " ")
        # fix "1 5" -> "15"
        day_str = re.sub(r'(?<=\d)\s+(?=\d)', '', day_str)
        # allow “15” or “15th”
        mday = re.search(
            r"(?i)(monday|tuesday|wednesday|thursday|friday|saturday|sunday)\s+(\d{1,2})(?:st|nd|rd|th)?\s+([A-Za-z]+)",
            day_str,
        )
        if not mday:
            continue
        _, daynum, mon = mday.groups()
        day_hint = datetime.strptime(f"{daynum} {mon} {YEAR_HINT}", "%d %B %Y").date()

        # Inside each day, find every group-block with sessions
        for tbl in day_div.select("table.tb-group-block.tb-events-block"):
            all_rows.extend(parse_group_sessions(tbl, day_hint))

        # Also handle keynotes (similar structure, different class)
        for tbl in day_div.select("table.tb-keynote-block.tb-events-block"):
            all_rows.extend(parse_group_sessions(tbl, day_hint))

    # Drop rows missing titles
    all_rows = [r for r in all_rows if r.get("title")]
    return all_rows

StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 41, Finished, Available, Finished)

In [40]:
rows = extract_schedule(START_URL)
print(f"Parsed {len(rows)} sessions")

df = pd.DataFrame(rows)

display(df)

StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 42, Finished, Available, Finished)

Parsed 158 sessions


SynapseWidget(Synapse.DataFrame, 739d86c7-af10-411b-a4ae-0c16032d81bc)

### put it together

In [41]:
df["session_id"] = CONF_CODE + df.index
df["conf_id"] = CONF_CODE
df["conference"] = CONFERENCE
display(df)

StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 43, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3b2485aa-98d4-4c49-a3d2-3ae2da5efebc)

In [42]:
from pyspark.sql import SparkSession, functions as F
spark = SparkSession.builder.getOrCreate()

sdf = spark.createDataFrame(df) \
    .withColumn("start", F.col("start").cast("timestamp")) \
    .withColumn("end",   F.col("end").cast("timestamp"))

# sdf.printSchema()


StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 44, Finished, Available, Finished)

In [43]:
display(sdf)

StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 45, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 413f3774-40a9-46d8-8e8c-aaa9d398b9f4)

### Get speakers

In [44]:
from pyspark.sql import functions as F

# Step 1: Split speakers column into an array
sessions_exploded = (
    sdf
    .withColumn("speaker_array", F.split(F.col("speakers"), ";"))
    .withColumn("speaker_raw", F.explode("speaker_array"))
    .withColumn("speaker_raw", F.trim(F.col("speaker_raw")))
    .drop("speaker_array")
)

sessions_exploded = (
    sessions_exploded
    .withColumn("speaker_name", F.trim(F.split(F.col("speaker_raw"), ",")[0]))
    .withColumn("speaker_affiliation", F.trim(F.expr("substring(speaker_raw, instr(speaker_raw, ',')+1, length(speaker_raw))")))
)

final_speakers_df = sessions_exploded.select(
    "session_id",
    "speaker_name",
    "speaker_affiliation"
)

display(final_speakers_df)


StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 46, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 48485d35-d05f-496f-9e59-04cde4a46cda)

### Get session descriptions

In [45]:
import html
import re
def get_session_paragraphs(url, retry_ignorenitro=True, timeout=20):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
    }
    def fetch(u):
        r = requests.get(u, headers=headers, timeout=timeout)
        r.raise_for_status()
        return r.text

    def clean(txt):
        txt = html.unescape(txt)
        txt = re.sub(r"\s+", " ", txt).strip()
        return txt

    def good_text(txt):
        if not txt:
            return False
        low = txt.lower()
        if "share this on" in low:
            return False
        return len(txt.strip()) >= 10

    # fetch and parse
    html_text = fetch(url)
    soup = BeautifulSoup(html_text, "html.parser")
    for tag in soup(["script", "style", "noscript", "template"]):
        tag.decompose()

    selectors = [
        "section.entry-content p",
        "[itemprop='articleBody'] p",
        "article .entry-content p",
        "div.entry-content p",
    ]
    for sel in selectors:
        ps = soup.select(sel)
        texts = [clean(p.get_text(" ", strip=True)) for p in ps if good_text(p.get_text())]
        if texts:
            return texts

    # fallback: all p’s
    ps = soup.find_all("p")
    texts = [clean(p.get_text(" ", strip=True)) for p in ps if good_text(p.get_text())]
    if texts:
        return texts

    # try once with ?ignorenitro=1
    if retry_ignorenitro and "ignorenitro" not in url:
        sep = "&" if "?" in url else "?"
        try:
            return get_session_paragraphs(url + f"{sep}ignorenitro=1", retry_ignorenitro=False)
        except Exception:
            pass

    return []

StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 47, Finished, Available, Finished)

In [23]:
## Test description
## get_session_paragraphs("https://www.sharepointeurope.com/events/getting-started-with-microsoft-fabric/")

StatementMeta(, db5b92b8-61c4-4e5f-b8e0-0db9e7f23965, 25, Finished, Available, Finished)

['This half-day workshop provides a hands-on introduction to Microsoft Fabric, equipping attendees with foundational knowledge of its core workloads, including Data Factory, Data Warehouse, Data Engineering, Data Science, Real-Time Intelligence, Power BI, and Partner & Industry Workloads.',
 'Participants will also explore key enabling technologies such as Copilot, OneLake, and Purview, ensuring they understand how Fabric’s unified analytics platform supports scalable, data-driven solutions.',
 'Guided by seasoned professionals, attendees will work through hands-on exercises designed to solidify their learning, ask questions in real time, and gain practical experience using Fabric’s powerful capabilities.',
 'Participants will leave not only with a strong grasp of Fabric’s essential features but also with the confidence to maximize their conference experience and apply their learning to real-world scenarios.',
 'NOTE: This tutorial is a half day tutorial.',
 'When booking your ticket, 

In [46]:
import time
descs = []
for _, row in df.iterrows():
    sid = row["session_id"]
    url = row["detail_url"]
    try:
        paras = get_session_paragraphs(url) if url else []
        joined = " ".join(paras) if paras else None
    except Exception as e:
        print(f"Error on {sid}: {e}")
        joined = None
    descs.append({"session_id": sid, "description": joined})
    time.sleep(1.0)  # be polite

descs_pd = pd.DataFrame(descs)
descs_pd = descs_pd.drop_duplicates(subset=["session_id"])

# 4) Back to Spark and merge with original
descs_sdf = spark.createDataFrame(descs_pd)
sessions_with_desc = sdf.join(descs_sdf, on="session_id", how="left")

StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 48, Finished, Available, Finished)

In [47]:
display(descs_pd)

StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 49, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d0f51fb0-540f-4e33-9562-4677fc680864)

In [48]:
display(sessions_with_desc)

StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 50, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 408d131f-5fe0-42c2-a4ce-0d0b35750534)

### Save to tables

In [49]:
sessions_with_desc.write.format("delta").mode("overwrite").option("mergeSchema","true").saveAsTable("raw_" + CONF_PREFIX + "_sessions")

StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 51, Finished, Available, Finished)

In [50]:
final_speakers_df.write.format("delta").mode("overwrite").option("mergeSchema","true").saveAsTable("raw_" + CONF_PREFIX + "_speakers")

StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 52, Finished, Available, Finished)

### Write out csvs

In [51]:
p_sessions_with_desc = sessions_with_desc.toPandas() 
p_sessions_with_desc.to_csv("/lakehouse/default/Files/raw_" + CONF_PREFIX + "_sessions.csv", index=False) 

StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 53, Finished, Available, Finished)

In [52]:
p_speakers = final_speakers_df.toPandas() 
p_speakers.to_csv("/lakehouse/default/Files/raw_" + CONF_PREFIX + "_speakers.csv", index=False) 

StatementMeta(, c593e6a8-025d-426c-808e-d099c8a88ab0, 54, Finished, Available, Finished)